# In vitro cytotoxicity assay

##### Load the CSV file into a DataFrame and reading the data

In [ ]:
import pandas as pd

df = pd.read_csv("In vitro cytotoxicity assay.csv")
df

##### Data analysis

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd

data = {
    'Nanoparticle concentration(mg/ml)': df['Nanoparticle concentration(mg/ml)'],
    'A': df.iloc[0, 1:].values,
    'B': df.iloc[1, 1:].values,
    'C': df.iloc[2, 1:].values,
    'D': df.iloc[3, 1:].values,
    'E': df.iloc[4, 1:].values,
    'F': df.iloc[5, 1:].values,
    'G': df.iloc[6, 1:].values,
    'H': df.iloc[7, 1:].values,
    'I': df.iloc[8, 1:].values,
    'J': df.iloc[9, 1:].values,
    'K': df.iloc[10, 1:].values

}

rows = []
for conc, values in zip(data['Nanoparticle concentration(mg/ml)'], zip(data['A'], data['B'], data['C'], data['D'], data['E'], data['F'], data['G'], data['H'], data['I'], data['J'], data['K'])):
    for group, value in zip(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K'], values):
        rows.append({
            'Nanoparticle concentration(mg/ml)': conc,
            'Group': group,
            'Values': value
        })

df2 = pd.DataFrame(rows)

# One-way ANOVA
model = ols('Values ~ C(Group)', data=df2).fit()

anova_table = sm.stats.anova_lm(model, typ=1)
print("ANOVA Table:")
print(anova_table)

# Tukey's test
tukey = pairwise_tukeyhsd(endog=df2['Values'], groups=df2['Group'], alpha=0.05)
print("\nTukey's HSD Test:")
print(tukey)


##### Plot the results

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from comparison import pairwise_comparison

df['Mean_Cell_viability'] = df[['Cell viability1(%)', 'Cell viability2(%)', 'Cell viability3(%)']].mean(axis=1)

df['SEM'] = df[['Cell viability1(%)', 'Cell viability2(%)', 'Cell viability3(%)']].sem(axis=1)

plt.figure(figsize=(12, 8))
bar_width = 0.6
x_ticks = np.arange(len(df))

plt.bar(x_ticks, df['Mean_Cell_viability'], width=bar_width, color='cornflowerblue', edgecolor='darkslategray', linewidth=4)

errorbar_caps = plt.errorbar(x_ticks, df['Mean_Cell_viability'], yerr=df['SEM'], capsize=9, color='darkslategray', ecolor='darkslategray', markersize=0, fmt='_', elinewidth=2)

for cap in errorbar_caps[1]:
    cap.set_markeredgewidth(2)
    cap.set_path_effects([path_effects.Stroke(linewidth=2), path_effects.Normal()])

font_properties = {'fontweight': 'bold', 'fontsize': 25, 'fontfamily': 'Arial'}
number_font_properties = {'fontweight': 'bold', 'fontsize': 25, 'fontfamily': 'Arial'}

xtick_labels = ['Control', '0.75', '1.5', '3.12', '6.25', '12.5', '25', '50', '100', '500', '1000']
plt.xticks(x_ticks, xtick_labels, rotation=45, rotation_mode='anchor', ha='right', **number_font_properties)
plt.xlabel('Nanoparticle concentration(mg/ml)', **font_properties)

plt.yticks([0, 20, 40, 60, 80, 100], **number_font_properties)
plt.ylabel('Cell viability (%)', **font_properties)

plt.tick_params(axis='both', which='both', length=15, width=4, labelsize=25)

ax = plt.gca()
ax.spines['left'].set_linewidth(4)
ax.spines['bottom'].set_linewidth(4)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

pairwise_comparison(1, 3, 0.0004, x_ticks, df['Mean_Cell_viability'].tolist(), yerr=df['SEM'].tolist(), barh=0.015, fs=18, bold=True, line_width=2)
pairwise_comparison(3, 6, 0.0075, x_ticks, df['Mean_Cell_viability'].tolist(), yerr=df['SEM'].tolist(), dh= 0.085,barh=0.015, fs=18, bold=True, line_width=2)
pairwise_comparison(5, 7, 0.0183, x_ticks, df['Mean_Cell_viability'].tolist(), yerr=df['SEM'].tolist(), barh=0.015, fs=18, bold=True, line_width=2)
pairwise_comparison(7, 8, 0, x_ticks, df['Mean_Cell_viability'].tolist(), yerr=df['SEM'].tolist(), barh=0.015, fs=18, bold=True, line_width=2)
pairwise_comparison(9, 10, 0.2186, x_ticks, df['Mean_Cell_viability'].tolist(), yerr=df['SEM'].tolist(), barh=0.015, fs=15, bold=True, line_width=2)

plt.ylim(bottom=0)
plt.savefig('In_vitro_cytotoxicity_assay_graph.tif', dpi=600, format='tif', bbox_inches='tight')
plt.show()